In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import geopandas as gpd
# from ftplib import FTP
# import hvplot.xarray
import plotly.express as px
import calendar

In [ ]:
def get_min_max(df, parameter_name):
    param_data = df[df['parameter'] == parameter_name]
    if not param_data.empty:
        min_val = param_data['min'].values[0]
        max_val = param_data['max'].values[0]
        return min_val, max_val
    else:
        return None, None 

In [ ]:
df = pd.read_csv(r"D:\thesis_data\random_csv_output\complete_df.csv")

df

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# Plot histogram for df
fig1 = px.histogram(df, x='ndsi', nbins=100)
fig1.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

# # Plot histogram for df_to_delete
# fig2 = px.histogram(df, x='ndsi', nbins=100)
# fig2.update_layout(
#     xaxis_title='Value',
#     yaxis_title='Frequency',
# )

# # Create subplots with two columns
# fig = make_subplots(rows=1, cols=2, subplot_titles=('Before', 'After'))
# fig.add_trace(fig1.data[0], row=1, col=2)
# fig.add_trace(fig2.data[0], row=1, col=1)

# fig.update_layout(
#     title='Before and after zero sampling in mid-year months',
#     width=1000,  # Total width of the combined plots
#     height=400,  # Height of the combined plots
# )

# plotly_image_path = os.path.join(folder_name, 'before_after_zero_sampling.png')
# fig.write_image(plotly_image_path)


fig1.show()


In [ ]:
df

In [ ]:

# Assuming df is your dataframe with a column 'ndsi' containing NDSI values
# Make sure your dataframe has a column 'date' with datetime format

# Step 1: Identify months from April to November
selected_months = df[(df['Month'] >= 4) & (df['Month'] <= 11)]
selected_months


In [ ]:

non_selected_months = df.drop(selected_months.index)
non_selected_months

In [ ]:

# Step 2: Identify rows with NDSI values in the specified range
near_zero_rows = selected_months[(selected_months['ndsi'] >= -0.005) & (selected_months['ndsi'] <= 0.005)]
near_zero_rows


In [ ]:
# Step 3: Calculate the number of zero values you want to achieve (a)
a = 4000

# Step 4: Calculate the total number of rows in each month
total_rows_per_month = near_zero_rows.groupby('Month').size()
total_rows_per_month


In [ ]:
# Step 5: Calculate the percentage (b) to sample for each month
b = (a / total_rows_per_month).mean()
b

In [ ]:

# Step 6: Use the calculated percentage to sample the dataframe for each month
sampled_data = selected_months.groupby('Month',group_keys=False).apply(lambda x: x.sample(frac=b, replace=True))
sampled_data


In [ ]:
non_selected_months

In [ ]:
# Drop the 'month' column if you added it

sampled_data
# Now 'sampled_data' contains the sampled rows for each month with the desired number of zero values


In [ ]:
concatenated_df = pd.concat([sampled_data, non_selected_months], ignore_index=False)
concatenated_df

In [ ]:
concatenated_df['Date'] = pd.to_datetime(concatenated_df['Date'])

# Sort the DataFrame by the 'date' column
concatenated_df = concatenated_df.sort_values(by='Unnamed: 0')
concatenated_df

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# Plot histogram for df
fig1 = px.histogram(non_selected_months, x='ndsi', nbins=100)
fig1.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

# Plot histogram for df_to_delete
fig2 = px.histogram(df, x='ndsi', nbins=100)
fig2.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

# Create subplots with two columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Before', 'After'))
fig.add_trace(fig1.data[0], row=1, col=2)
fig.add_trace(fig2.data[0], row=1, col=1)

fig.update_layout(
    title='Before and after zero sampling in mid-year months',
    width=1000,  # Total width of the combined plots
    height=400,  # Height of the combined plots
)

# plotly_image_path = os.path.join(folder_name, 'before_after_zero_sampling.png')
# fig.write_image(plotly_image_path)


fig.show()


In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import geopandas as gpd
# from ftplib import FTP
# import hvplot.xarray
import plotly.express as px
import calendar


In [ ]:
#bookmark
df = pd.read_csv(r"D:\thesis_data\random_csv_output\complete_df.csv")
df['Date'] = pd.to_datetime(df['Date'])
df

In [ ]:
number_of_zero_array = []
percentage_to_sample_array = []

In [ ]:
# Step 1: Set the initial values
a = 30000  # Desired number of zero values
a = a - 2500 
tolerance = 0.05 * a  # Tolerance level for convergence
b = 0.01  # Initial percentage to sample
print("Tolerance: ", tolerance)
print(f"Final range of near zeores: {a +2500 - tolerance} - {a + 2500 + tolerance}")

In [ ]:
high_zero_month_df = df[df['Month'].isin([1, 2, 3, 12])]
while True:
    zero_values_array = []

    # Step 2: Iterate through each month
    for month in range(4, 12):  # Months from April to November
        # Step 3: Sample rows and calculate the number of near zero values
        sampled_data = df[(df['Date'].dt.month == month)].sample(frac=b)
        zero_values_array.append(np.sum((sampled_data['ndsi'] >= -0.005) & (sampled_data['ndsi'] <= 0.005)))

    # Step 4: Check the sum of zero values
    total_zero_values = sum(zero_values_array)

    # Step 5: Adjust the percentage (b)
    if total_zero_values > a + tolerance:
        b -= 0.01
    elif total_zero_values < a - tolerance:
        b += 0.01
    else:
        break  # Convergence reached

# Step 6: Display the final value of b
print(f"Final value of b: {b}")
print(f"Total zero values: {total_zero_values+2500}")


In [ ]:

# Now you can use this final value of b to sample your DataFrame
final_sampled_data = pd.concat([df[(df['Date'].dt.month == month)].sample(frac=b) for month in range(4, 12)])
final_sampled_data = pd.concat([final_sampled_data, high_zero_month_df])
final_sampled_data = final_sampled_data.sort_values("Unnamed: 0")
final_sampled_data

In [ ]:

# Plot histogram for df
fig1 = px.histogram(df, x='ndsi', nbins=100)
fig1.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

# Plot histogram for df_to_delete
fig2 = px.histogram(final_sampled_data, x='ndsi', nbins=100)
fig2.update_layout(
    xaxis_title='Value',
    yaxis_title='Frequency',
)

# Create subplots with two columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Before', 'After'))
fig.add_trace(fig1.data[0], row=1, col=2)
fig.add_trace(fig2.data[0], row=1, col=1)

fig.update_layout(
    title='Before and after zero sampling in mid-year months',
    width=1000,  # Total width of the combined plots
    height=400,  # Height of the combined plots
)

# plotly_image_path = os.path.join(folder_name, 'before_after_zero_sampling.png')
# fig.write_image(plotly_image_path)


fig.show()


In [ ]:
df['Date'], scale = df['Date'].dt.strftime('%m-%Y'), "Average monthly"
df= df.groupby('Date')[["ndsi"]].mean().reset_index()
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')

fig = px.line(df, x="Date" , y=["ndsi"])
fig.update_layout(title=f'Original', width = 1800, height = 400)

fig

In [ ]:
final_sampled_data['Date'], scale = final_sampled_data['Date'].dt.strftime('%m-%Y'), "Average monthly"
final_sampled_data= final_sampled_data.groupby('Date')[["ndsi"]].mean().reset_index()
final_sampled_data['Date'] = pd.to_datetime(final_sampled_data['Date'])
final_sampled_data = final_sampled_data.sort_values(by='Date')

fig = px.line(final_sampled_data, x="Date" , y=["ndsi"])
fig.update_layout(title=f'Original', width = 1800, height = 400)

fig

In [ ]:


# Assuming your DataFrame is named 'df'
# Replace this with your actual DataFrame if it's named differently

# Create a sample DataFrame for demonstration
data = {
    'ndsi': [0.1, 0.5, 0.2, 0.8, 0.4, 0.6, 0.3, 0.9, 0.7, 0.5],
    'col1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    # Add other columns as needed
}

df = pd.DataFrame(data)
df

In [ ]:
df['ndsi'].sum()

In [ ]:

# Calculate the weights based on the 'ndsi' column
weights = df['ndsi'] / df['ndsi'].sum()
weights


In [ ]:
# Sample 10% of the DataFrame using the calculated weights
sampled_df = df.sample(frac=0.1, replace=True, weights=weights)
sampled_df


In [ ]:

# Reset the index of the sampled DataFrame if needed
sampled_df.reset_index(drop=True, inplace=True)

# Print the sampled DataFrame
print(sampled_df)


#old stuff, i didnt botehr

In [ ]:
min_max_df = pd.read_csv(r"D:\thesis_data\random_csv_output\min_max_of_all_parameters.csv")
min_max_df

In [ ]:
parameter_array = ["tas", "pr", "huss", "rlds",  "rsds", "sfcWind"]
# parameter_array = ["tas"]


# min_max_df = pd.read_csv("./min_max_of_all_parameters.csv")

# applying the min max scaling
for parameter in parameter_array:
    min_val, max_val =  get_min_max(min_max_df, parameter)
    df[parameter] = (df[parameter] - min_val) / (max_val - min_val)

df['ndsi'] = df['ndsi'].apply(lambda x: x / 100 if not pd.isnull(x) else x)

df = df.dropna()
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# df

In [ ]:
df_to_delete

In [ ]:
fig = px.histogram(df_to_delete, x='ndsi', nbins=100)

# Update layout for better visibility (optional)
fig.update_layout(title='Interactive Histogram of Column',
                  xaxis_title='Value',
                  yaxis_title='Frequency')

In [ ]:
month_fraction_array = [0.357142857, 0.117647059, 0.052631579, 0.03030303, 0.028571429, 0.035714286, 0.0625, 0.285714286]

month_fraction_array = [round(x, 2) for x in month_fraction_array]

# Display the rounded array
print(month_fraction_array)
# it's a copy of the last cell's code. Just editing in it to make it work.
# Getting the 1. Number of zeroes 2. Number of near zeroes 3. Number of NaNs in the ndsi data
month_number = 12
year = 2018

year_array = []
total_len_array = []
zero_array = []
more_than_zero_array = []
nan_count_array = []

for i in range(4, 12):
    df_zero_ndsi = df_to_delete
    j = i-4
    df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Month'] == i]
    # df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Year'].isin([i])]  # will come back to this later
    df_zero_ndsi = df_zero_ndsi.sample(frac=month_fraction_array[j])
    
    
    # 3. now add it back to the original dataframe
    # combined_df =  pd.concat([df1, df2], ignore_index=True)
    df =  pd.concat([df, df_zero_ndsi], ignore_index=True)

df_zero_ndsi

In [ ]:
df

In [ ]:
fig = px.histogram(df, x='ndsi', nbins=100)

# Update layout for better visibility (optional)
fig.update_layout(title='Interactive Histogram of Column',
                  xaxis_title='Value',
                  yaxis_title='Frequency')

# look down you'll find something, I was jsut lazy here

In [ ]:
px.histogram(df_zero_ndsi, x='ndsi', nbins=100)

In [ ]:
fig = px.line(comparison_df, x="Date" , y=["y_pred","y_test"])
fig.update_layout(title=f'Predicted vs Observed | {scale}')

In [ ]:
# it's a copy of the last cell's code. Just editing in it to make it work.
# Getting the 1. Number of zeroes 2. Number of near zeroes 3. Number of NaNs in the ndsi data
month_number = 12
year = 2018

year_array = []
total_len_array = []
zero_array = []
more_than_zero_array = []
nan_count_array = []

for i in range(4, 12):
    df_zero_ndsi = df_to_delete
    df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Month'] == i]
    # df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Year'].isin([i])]  # will come back to this later
    
    # number_of_zero = df_zero_ndsi['check'].sum()
    number_of_more_than_zero = df_zero_ndsi['check2'].sum()

    
    #1. name a df temp and only select the ones which have check = 0, 
    temp = df_zero_ndsi 
    df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['check'] == 0]

    # 2. now randomly select 1.5*number_of_more_than_zero in it
    temp = temp[temp['check'] == 1]
    temp = temp.sample(n = int(1.5*number_of_more_than_zero))

    # 3. now add it back to the original dataframe
    # combined_df =  pd.concat([df1, df2], ignore_index=True)
    df =  pd.concat([df, temp, df_zero_ndsi], ignore_index=True)

df_zero_ndsi

In [ ]:
temp

In [ ]:
combined_df= df
combined_df = combined_df.sort_values(by='Date', ascending=True)
combined_df = combined_df[["Date", "Latitude", "Longitude", "ndsi", "Month", "Year", "check", "check2"]]
combined_df

In [ ]:
combined_df, scale = combined_df.sort_values(by='Date'), "Original without mean"
fig = px.scatter(combined_df, x="Date" , y="ndsi")
fig.update_layout(title=f'Predicted vs Observed | {scale}')


In [ ]:


combined_df_1 = combined_df[(combined_df['Latitude'] ==  46.58) & (combined_df['Longitude'] == 11.15)]
combined_df_1, scale = combined_df_1.sort_values(by='Date'), "Centre without mean"
fig = px.scatter(combined_df, x="Date" , y="ndsi")
fig.update_layout(title=f'Predicted vs Observed | {scale}')


In [ ]:
combined_df_daily_mean = combined_df
combined_df_daily_mean['Date'] = pd.to_datetime(combined_df_daily_mean['Date'], format='%d-%m-%Y')
combined_df_daily_mean= combined_df_daily_mean.groupby('Date')[["ndsi"]].mean().reset_index()
combined_df_daily_mean = combined_df_daily_mean.sort_values(by='Date')
# combined_df_daily_mean
fig = px.scatter(combined_df_daily_mean, x="Date" , y="ndsi")
fig.update_layout(title=f'Predicted vs Observed | {scale}')


In [ ]:
combined_df_monthly = combined_df
combined_df_monthly['Date'] = pd.to_datetime(combined_df_monthly['Date'])
combined_df_monthly = combined_df_monthly.sort_values(by='Date')
combined_df_monthly['Date'], scale = combined_df_monthly['Date'].dt.strftime('%m-%Y'), "Average monthly"
combined_df_monthly= combined_df_monthly.groupby('Date')[["ndsi"]].mean().reset_index()
combined_df_monthly['Date'] = pd.to_datetime(combined_df_monthly['Date'])
combined_df_monthly = combined_df_monthly.sort_values(by='Date')

fig = px.line(combined_df_monthly, x="Date" , y="ndsi")
fig.update_layout(title=f'Predicted vs Observed | {scale}')



In [ ]:
# Create an interactive histogram using Plotly
fig = px.histogram(combined_df, x='ndsi', nbins=100)

# Update layout for better visibility (optional)
fig.update_layout(title='Interactive Histogram of Column',
                  xaxis_title='Value',
                  yaxis_title='Frequency')

In [ ]:

# # Getting the 1. Number of zeroes 2. Number of near zeroes 3. Number of NaNs in the ndsi data
# month_number = 12
# year = 2018

# year_array = []
# total_len_array = []
# zero_array = []
# more_than_zero_array = []
# nan_count_array = []

# for i in range(2001, 2019):
#     year_array.append(i)    
#     df['Date'] = pd.to_datetime(df['Date'])
#     df["Month"] = df['Date'].dt.month
#     df["Year"] = df['Date'].dt.year

#     df_zero_ndsi = df
#     df_zero_ndsi = df_zero_ndsi.dropna()
#     df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Month'] == month_number]
#     df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Year'].isin([i])]
    
#     number_of_na = (calendar.monthrange(i, month_number)[1]*140) - len(df_zero_ndsi)
#     nan_count_array.append(number_of_na)
    
#     # total_len_array.append(len(df_zero_ndsi))

    
#     df_zero_ndsi['check'] = np.where((df_zero_ndsi['ndsi'] >= -0.005) & (df_zero_ndsi['ndsi'] <= 0.005), 1, 0)
#     df_zero_ndsi['check2'] = np.where((df_zero_ndsi['ndsi'] > 0.005) & (df_zero_ndsi['ndsi'] <= 0.015), 1, 0)
#     # df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['ndsi'] != 105]

    
#     number_of_zero = df_zero_ndsi['check'].sum()
#     number_of_more_than_zero = df_zero_ndsi['check2'].sum()

#     zero_array.append(number_of_zero)
#     more_than_zero_array.append(number_of_more_than_zero)

#     # df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['check2'] == 1]
#     # df_zero_ndsi = df_zero_ndsi[~df_zero_ndsi['Month'].isin([1, 2, 3, 4, 11, 12])]

# # df_zero_ndsi= df_zero_ndsi.drop(["Month", "check"],axis =1)
# # df = df_zero_ndsi 
    
# df_zero_ndsi

In [ ]:
# # Getting the 1. Number of zeroes 2. Number of near zeroes 3. Number of NaNs in the ndsi data
# month_number = 12
# year = 2018

# year_array = []
# total_len_array = []
# zero_array = []
# more_than_zero_array = []
# nan_count_array = []

# for i in range(2001, 2019):
#     year_array.append(i)    
#     df['Date'] = pd.to_datetime(df['Date'])
#     df["Month"] = df['Date'].dt.month
#     df["Year"] = df['Date'].dt.year

#     df_zero_ndsi = df
#     df_zero_ndsi = df_zero_ndsi.dropna()
#     df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Month'] == month_number]
#     df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['Year'].isin([i])]
    
#     number_of_na = (calendar.monthrange(i, month_number)[1]*140) - len(df_zero_ndsi)
#     nan_count_array.append(number_of_na)
    
#     # total_len_array.append(len(df_zero_ndsi))

    
#     df_zero_ndsi['check'] = np.where((df_zero_ndsi['ndsi'] >= -0.005) & (df_zero_ndsi['ndsi'] <= 0.005), 1, 0)
#     df_zero_ndsi['check2'] = np.where((df_zero_ndsi['ndsi'] > 0.005) & (df_zero_ndsi['ndsi'] <= 0.015), 1, 0)
#     # df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['ndsi'] != 105]

    
#     number_of_zero = df_zero_ndsi['check'].sum()
#     number_of_more_than_zero = df_zero_ndsi['check2'].sum()

#     zero_array.append(number_of_zero)
#     more_than_zero_array.append(number_of_more_than_zero)

#     # df_zero_ndsi = df_zero_ndsi[df_zero_ndsi['check2'] == 1]
#     # df_zero_ndsi = df_zero_ndsi[~df_zero_ndsi['Month'].isin([1, 2, 3, 4, 11, 12])]

# # df_zero_ndsi= df_zero_ndsi.drop(["Month", "check"],axis =1)
# # df = df_zero_ndsi 
    
# df_zero_ndsi

In [ ]:
dict_temp = {
    f"Year {calendar.month_name[month_number]}": year_array,
    "Number of zeroes": zero_array,
    "Slightly more than zeroes": more_than_zero_array,
    # "Total rows": total_len_array,
    "Number of NaNs": nan_count_array
}

# Create the DataFrame
zero_analysis_df = pd.DataFrame(dict_temp)
zero_analysis_df

In [ ]:

# Create an interactive histogram using Plotly
fig = px.histogram(df_zero_ndsi, x=['ndsi'] , nbins=100)

# Update layout for better visibility (optional)
fig.update_layout(title=f'{calendar.month_name[month_number]}',
                  xaxis_title='Value',
                  yaxis_title='Frequency')

In [ ]:
import matplotlib.pyplot as plt

# Grouping by Latitude and Longitude, calculating the average 'ndsi' for each group
avg_ndsi = df_zero_ndsi.groupby(['Latitude', 'Longitude'])['ndsi'].mean().reset_index()

# Creating a grid for latitude and longitude
latitudes = avg_ndsi['Latitude'].unique()
longitudes = avg_ndsi['Longitude'].unique()
grid_ndsi = np.zeros((len(latitudes), len(longitudes)))

# Filling the grid with average 'ndsi' values
for index, row in avg_ndsi.iterrows():
    lat_idx = np.where(latitudes == row['Latitude'])[0][0]
    lon_idx = np.where(longitudes == row['Longitude'])[0][0]
    grid_ndsi[lat_idx, lon_idx] = row['ndsi']

# Plotting the grid data
plt.figure(figsize=(8, 6))
plt.imshow(grid_ndsi, extent=[longitudes.min(), longitudes.max(), latitudes.min(), latitudes.max()])
plt.colorbar(label='Average NDSI')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title(f'Average NDSI across Latitude and Longitude for {calendar.month_name[month_number]} of {year}')

plt.show()


In [ ]:
# Create an interactive histogram using Plotly
fig = px.histogram(df_zero_ndsi, x='Month', nbins=100)

# Update layout for better visibility (optional)
fig.update_layout(title='Number of zero NDSI values vs the number of month',
                  xaxis_title='Month',
                  yaxis_title='Number of zero NDSI')

In [ ]:
# fig = px.scatter(df, x="Date" , y=["ndsi"])
# fig